https://stackoverflow.com/questions/58568400/weighted-summation-of-embeddings-in-pytorch

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn

## Variable multi-categorica

Es una variable categorica comun con la diferencia que la columna en la observacion no tiene un solo valor sino una lista de valores.

Ej.: Las peliculas pertenecen a varios generos. El genero es una variable categorica comun pero las peliculas pueden petenecer mas de un genero. Entonces una observacion de una pelicula podria ser:

- Nombre: Toy Story
- Generos: [Comedia, Fantasticas, Aventura]

## Embedding Bag

* Un Embedding bag permite sumar, promediar(pesado o normal) o quedarnos con el maximo de una lista de embedding vectors.
* Son muy usados cuando tenemos una variable muti-categorica.
* Si queres usar un promedio pesado el problema es que los pesos no son parametros a aprender, si no que hay que pasarlos. Son parametros fijos :(.
* Lo mejor seria tener un EmbeddingBag que aprenda esos pesos ajustandolos en el proceso de backpropagation ;)

## Weighted Mean Embedding Bag

* A continuación se implementa un EmbeddingBag con promedio pesado, donde los pesos son parametros 
a apender por la capa (Módulo en pytorch).
* Se separa el problema es dos pasos. 
  * Una capa embedding comun que en base a los indices de las categorias devueve embedding vectors
  * Otra capa (**LinearWeightedAVG**) que toma estos vectores, hace el promedio pesado y se queda con un unico vector embedding promedio para cada batch.

In [3]:
def avg_layer(embeding_count):
    return nn.Conv1d(in_channels=embeding_count, out_channels=1, kernel_size=1)

class LinearWeightedMean(nn.Module):
    def __init__(self): 
        super(LinearWeightedMean, self).__init__()
        self._avg = None

    def forward(self, x): 
        if self._avg == None: self._avg = avg_layer(embeding_count = embed.size()[-2])
        return torch.stack([self._avg(batch) for batch in embed]).squeeze(-2)

class WeightedMeanEmbeddingBag(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(WeightedMeanEmbeddingBag, self).__init__()
        self._emb = embedding = nn.Embedding(num_embeddings, embedding_dim)
        self._avg = LinearWeightedMean()
 
    def forward(self, x): return self._avg(self._emb(x))

## Ejemplo

* Cada observación es una lista de categorias de una variable categorica codificadas en numeros.
* La variable categorica tiene 3 posibles valores, excluyentes en cada posición de la lista de valores. Por ej.: una pelicula no puede tener dos veces el genero comedia.
* Cada observacion es una lista de tamaño 3, por que una pelicula podria tener todos los generos posibles (3 en este ejemplo).
* Algunas peliculas pueden tener menos generos que el total. Los que faltantes quedan en cero.

In [4]:
embedding = nn.Embedding(
    num_embeddings=4, # La opcion sin genero es un valor mas de la categorica.
    embedding_dim=2
)
embedding.weight

Parameter containing:
tensor([[ 1.2157,  0.9606],
        [-0.5736,  0.7405],
        [-0.7917, -0.0940],
        [ 1.0521, -1.1056]], requires_grad=True)

In [5]:
input_ = torch.LongTensor([
    [ 
        [1, 2, 3], # La pelicula 1 tiene los generos 1,2 y 3.
        [3, 0, 0]  # La pelicula 2 tiene el generos 2 solamente.
    ],
    [ 
        [1, 2, 0], 
        [2, 0, 0]
    ]
])

input_.size()

torch.Size([2, 2, 3])

Tenemos un 2 lotes de 2 observaciones cada uno:

In [6]:
input_

tensor([[[1, 2, 3],
         [3, 0, 0]],

        [[1, 2, 0],
         [2, 0, 0]]])

In [7]:
embed = embedding(input_)

In [8]:
embed.size()

torch.Size([2, 2, 3, 2])

In [9]:
embed

tensor([[[[-0.5736,  0.7405],
          [-0.7917, -0.0940],
          [ 1.0521, -1.1056]],

         [[ 1.0521, -1.1056],
          [ 1.2157,  0.9606],
          [ 1.2157,  0.9606]]],


        [[[-0.5736,  0.7405],
          [-0.7917, -0.0940],
          [ 1.2157,  0.9606]],

         [[-0.7917, -0.0940],
          [ 1.2157,  0.9606],
          [ 1.2157,  0.9606]]]], grad_fn=<EmbeddingBackward0>)

In [10]:
embed.size()

torch.Size([2, 2, 3, 2])

In [11]:
avg = LinearWeightedMean()

In [12]:
out = avg(embed)
out

tensor([[[ 0.2593, -0.7678],
         [ 1.0706,  0.9285]],

        [[ 0.3533,  0.4193],
         [ 1.1610,  0.8789]]], grad_fn=<SqueezeBackward1>)

In [13]:
out.size()

torch.Size([2, 2, 2])

In [18]:
wmean = WeightedMeanEmbeddingBag(num_embeddings=4, embedding_dim=2)

In [19]:
out = wmean(input_)
out

tensor([[[0.3720, 0.8317],
         [0.2079, 0.2181]],

        [[0.3356, 0.3721],
         [0.1486, 0.2507]]], grad_fn=<SqueezeBackward1>)

In [20]:
wmean

WeightedMeanEmbeddingBag(
  (_emb): Embedding(4, 2)
  (_avg): LinearWeightedMean(
    (_avg): Conv1d(3, 1, kernel_size=(1,), stride=(1,))
  )
)

In [21]:
out.size()

torch.Size([2, 2, 2])